In [1]:
%dependency /add com.microsoft.semantic-kernel:semantickernel-api:1.0.0-rc2
%dependency /add com.microsoft.semantic-kernel:semantickernel-bom:1.0.0-rc2
%dependency /add com.microsoft.semantic-kernel:semantickernel-parent:1.0.0-rc2
%dependency /add com.microsoft.semantic-kernel:semantickernel-aiservices-openai:1.0.0-rc2
%dependency /add org.slf4j:slf4j-jdk14:2.0.12

%dependency /resolve

Adding dependency com.microsoft.semantic-kernel:semantickernel-api:1.0.0-rc2
Adding dependency com.microsoft.semantic-kernel:semantickernel-bom:1.0.0-rc2
Adding dependency com.microsoft.semantic-kernel:semantickernel-parent:1.0.0-rc2
Adding dependency com.microsoft.semantic-kernel:semantickernel-aiservices-openai:1.0.0-rc2
Adding dependency org.slf4j:slf4j-jdk14:2.0.12
Solving dependencies
Found dependencies count: 42
Add to classpath: /Users/lokinfey/Library/Jupyter/kernels/rapaio-jupyter-kernel/ivy_cache/com.microsoft.semantic-kernel/semantickernel-api/jars/semantickernel-api-1.0.0-rc2.jar
Add to classpath: /Users/lokinfey/Library/Jupyter/kernels/rapaio-jupyter-kernel/ivy_cache/com.azure/azure-ai-openai/jars/azure-ai-openai-1.0.0-beta.6.jar
Add to classpath: /Users/lokinfey/Library/Jupyter/kernels/rapaio-jupyter-kernel/ivy_cache/com.azure/azure-core/jars/azure-core-1.45.1.jar
Add to classpath: /Users/lokinfey/Library/Jupyter/kernels/rapaio-jupyter-kernel/ivy_cache/com.azure/azure-jso

In [ ]:

import com.microsoft.semantickernel.planner.sequentialplanner.SequentialPlanner;